In [1]:
import os
import time
import math
import gzip
import re
import numpy as np
import pandas as pd
import EmitGCL 
from EmitGCL.utils import *
from EmitGCL.loss_function import *
from EmitGCL.conv import *
from EmitGCL.emitgcl_model import *

In [2]:
def get_cancer_metastasis_genes():
    kegg = KEGG()

    # Define cancer metastasis pathways
    cancer_metastasis_pathways = {
        'Protein processing in endoplasmic reticulum': 'hsa04141',
        'mTOR signaling pathway': 'hsa04150',
        'NF-kappa B signaling pathway': 'hsa04064',
        'Autophagy': 'hsa04140',
        'p53 signaling pathway': 'hsa04115',
        'Apoptosis': 'hsa04210'
    }


    pathway_genes = {}

    # Iterate over each pathway
    for pathway_name, pathway_id in cancer_metastasis_pathways.items():
        # Retrieve pathway information
        pathway_info = kegg.get(pathway_id)
        parsed_pathway = kegg.parse(pathway_info)

        # Get and store the gene list
        genes = parsed_pathway['GENE']
        gene_symbols = []
        for gene_id, gene_info in genes.items():
            # Get the gene symbol
            gene_symbol = gene_info.split(' ')[0].split(';')[0]
            gene_symbols.append(gene_symbol)

        pathway_genes[pathway_name] = gene_symbols

    return pathway_genes

pathway_genes = get_cancer_metastasis_genes()

#### Construct input file (adata format)

In [3]:
import scanpy as sc
import pandas as pd
import scipy.io
import os
import numpy as np
from scipy.sparse import vstack, csr_matrix

data_dir = "/users/PCON0022/duanmaoteng/Metastasis/Stress/GSE251730/Rep2"

rep2_untreated_barcodes = os.path.join(data_dir, "GSM7986594_Rep2_S1_barcodes.tsv")
rep2_untreated_features = os.path.join(data_dir, "GSM7986594_Rep2_S1_features.tsv")
rep2_untreated_matrix = os.path.join(data_dir, "GSM7986594_Rep2_S1_matrix.mtx")

rep2_6hr_barcodes = os.path.join(data_dir, "GSM7986595_Rep2_S2_barcodes.tsv")
rep2_6hr_features = os.path.join(data_dir, "GSM7986595_Rep2_S2_features.tsv")
rep2_6hr_matrix = os.path.join(data_dir, "GSM7986595_Rep2_S2_matrix.mtx")

rep2_18hr_barcodes = os.path.join(data_dir, "GSM7986598_Rep2_S5_barcodes.tsv")
rep2_18hr_features = os.path.join(data_dir, "GSM7986598_Rep2_S5_features.tsv")
rep2_18hr_matrix = os.path.join(data_dir, "GSM7986598_Rep2_S5_matrix.mtx")

def load_data(matrix_file, barcodes_file, features_file):
    matrix = scipy.io.mmread(matrix_file).T.tocsr()

    barcodes = pd.read_csv(barcodes_file, header=None, sep='\t')

    features = pd.read_csv(features_file, header=None, sep='\t')

    adata = sc.AnnData(X=matrix)
    adata.obs['barcode'] = barcodes[0].values
    adata.var['gene'] = features[0].values
    adata.var['gene_name'] = features[1].values

    adata.obs_names = adata.obs['barcode']
    adata.var_names = adata.var['gene_name']

    return adata

adata_untreated = load_data(rep2_untreated_matrix, rep2_untreated_barcodes, rep2_untreated_features)

adata_6hr = load_data(rep2_6hr_matrix, rep2_6hr_barcodes, rep2_6hr_features)

adata_18hr = load_data(rep2_18hr_matrix, rep2_18hr_barcodes, rep2_18hr_features)

/tmp/slurmtmp.32559692/ipykernel_276659/1659835070.py:38: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(X=matrix)
/tmp/slurmtmp.32559692/ipykernel_276659/1659835070.py:38: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(X=matrix)
/tmp/slurmtmp.32559692/ipykernel_276659/1659835070.py:38: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(X=

#### Train model

In [10]:
import numpy as np
import pandas as pd
from scipy.sparse import vstack, csr_matrix
# Main execution
if __name__ == "__main__":
    
    input_directory = 'Rep2_IL-1b'

    ignore_warnings()
    set_random_seed()

    # Parses command-line arguments for training a GNN on a gene cell graph, you can modify it as needed.
    (output_file, labsm, lr, wd, n_hid, nheads, nlayers, 
     cell_size, neighbor, egrn) = parse_arguments(egrn=True, output_file= f'/users/PCON0022/duanmaoteng/Metastasis/Stress/GSE251730_Code/{input_directory}/')
    
    # Get the genes associated with the pathways
    pathway_genes = get_cancer_metastasis_genes()
    
    file_path = f'/users/PCON0022/duanmaoteng/Metastasis/Stress/GSE251730_Code/{input_directory}/'
    output_file = f'/users/PCON0022/duanmaoteng/Metastasis/Stress/GSE251730_Code/{input_directory}/Metastasis_Result_new/'
    attention_file = f'/users/PCON0022/duanmaoteng/Metastasis/Stress/GSE251730_Code/{input_directory}/Metastasis_Result_new/Attention/'
    ensure_dir(output_file)
    ensure_dir(attention_file)
    
    rep2_untreated_labels = ["Tumor"] * adata_untreated.shape[0]
    rep2_labeled_6hr = ["Lymph Node"] * adata_6hr.shape[0] 
    rep2_labeled_18hr = ["Lymph Node"] * adata_18hr.shape[0]
    labels = rep2_untreated_labels + rep2_labeled_6hr + rep2_labeled_18hr

    gene_names = list(adata_untreated.var_names)
    untreated_cell_names = list(adata_untreated.obs_names)
    cell_names = untreated_cell_names + list(adata_6hr.obs_names) + list(adata_18hr.obs_names)

    untreated_matrix = adata_untreated.X
    matrix_6hr = adata_6hr.X
    matrix_18hr = adata_18hr.X
    cell_counts_matrix = vstack([untreated_matrix, matrix_6hr, matrix_18hr])

    cell_counts_matrix_csr = csr_matrix(cell_counts_matrix)

    gene_cell = cell_counts_matrix_csr.transpose()

    gene_cell.obs_names = gene_names
    gene_cell.var_names = cell_names 

    RNA_matrix = gene_cell

    cell_num = RNA_matrix.shape[1]
    gene_num = RNA_matrix.shape[0]

    print(f"Cell number: {cell_num}")
    print(f"Gene number: {gene_num}")

    device = torch.device("cuda" if cuda.is_available() else "cpu")
    device = torch.device('cpu')

    print('You will use : ',device)
    # Clustering result by scanpy
    # initial_pre = initial_clustering(RNA_matrix,custom_resolution=0.5,custom_n_neighbors=10)
    initial_pre = initial_clustering(RNA_matrix.copy())
    cluster_ini_num = len(set(initial_pre))
    ini_p1 = [int(i) for i in initial_pre]

    # Call the batch_select_whole function
    indices, Node_Ids, dic = batch_select_whole(RNA_matrix, labels, neighbor=[neighbor], cell_size=cell_size)
    # indices, Node_Ids, dic = batch_select_whole(RNA_matrix, label)
    sample_type = list(np.array(labels)[Node_Ids])
    n_batch = len(indices)

Cell number: 8329
Gene number: 36601
You will use :  cpu
	When the number of cells is less than or equal to 500, it is recommended to set the resolution value to 0.2.
	When the number of cells is within the range of 500 to 5000, the resolution value should be set to 0.5.
	When the number of cells is greater than 5000, the resolution value should be set to 0.8.
         Falling back to preprocessing with `sc.pp.pca` and default params.
Partitioning data into batches based on sample type.


Processing Lymph Node samples: 100%|██████████| 196/196 [01:03<00:00,  3.09it/s]


In [11]:
n_batch = len(indices)

# Reduce the dimensionality of features for cell, gene, and peak data.
node_model = NodeDimensionReduction(RNA_matrix, indices, ini_p1, n_hid=n_hid, n_heads=nheads, 
                                n_layers=nlayers, labsm=labsm, lr=lr, wd=wd, device=device, 
                                num_types=2, num_relations=1, epochs=10)
gnn = node_model.train_model(n_batch=n_batch)

# Tarin EmitGCL Model
EmitGCL_model = EmitGCL(gnn=gnn, labsm=labsm, n_hid=n_hid, n_batch=n_batch, device=device, lr=lr, wd=wd, 
                  pathway_genes=pathway_genes, gene_names=gene_names, sample_type=sample_type, num_epochs=10)
EmitGCL_gnn = EmitGCL_model.train_model(indices=indices, RNA_matrix=RNA_matrix, ini_p1=ini_p1, sample_type=sample_type, nodes_id=Node_Ids)

EmitGCL_result = EmitGCL_pred(RNA_matrix, EmitGCL_gnn=EmitGCL_gnn, indices=indices, nheads=nheads,
                            nodes_id=Node_Ids, cell_size=cell_size, device=device, 
                            gene_names=gene_names, node_dim_reduction_model=node_model)

attention_matrices = EmitGCL_result['attention_weights']

The training process for the NodeDimensionReduction model has started. Please wait.


 10%|█         | 1/10 [04:12<37:52, 252.50s/it]

Epoch 1:
  KL Loss: 0.22311657453523315
  Cluster Loss: 3.603064639593965


 20%|██        | 2/10 [08:31<34:11, 256.47s/it]

Epoch 2:
  KL Loss: 0.1949441485827969
  Cluster Loss: 3.0310178264494865


...

100%|██████████| 279/279 [04:12<00:00,  1.11it/s]


Epoch 9:
  KL Loss: 0.4730638861656189
  Cluster Loss: 2.328864574432373
  UCell Loss: -3.554826021194458
  Total Contrastive Loss: 1.4789232015609741
  Total Loss: 0.7260257005691528


100%|██████████| 279/279 [04:16<00:00,  1.09it/s]


Epoch 10:
  KL Loss: 0.4703826308250427
  Cluster Loss: 2.3217310905456543
  UCell Loss: -3.8056516647338867
  Total Contrastive Loss: 1.393241286277771
  Total Loss: 0.3797034025192261
The training for the EmitGCL model has been completed.


100%|██████████| 279/279 [07:12<00:00,  1.55s/it]


In [12]:
# Save Result
for head, matrix in attention_matrices.items():
    save_attention_matrix_to_mtx(matrix, attention_file, head)

np.save(output_file + "Node_Ids.npy", Node_Ids)
np.save(output_file + "pred.npy", EmitGCL_result['pred_label'])
np.save(output_file + "cell_embedding.npy", EmitGCL_result['cell_embedding'])